In [1]:
import pandas as pd
import plotly.graph_objects as go
from plotly.subplots import make_subplots

In [2]:
evc_df = pd.read_csv('../data/processed/evc_df.csv')
vfacts_df = pd.read_csv('../data/processed/vfacts_df.csv')
nsw_df = pd.read_csv('../data/processed/nsw_rego_data.csv')

In [3]:
vt = vfacts_df.loc[vfacts_df['State']=='NSW'].groupby(['Fuel Type']).sum().drop(columns=['State','Vehicle Type'])
vt = vt/vt.sum()*100
vte = vt.loc['Electric']
# plotly line plot of vte with x-axis as datetime and y-axis as vte
fig = go.Figure()
fig.add_trace(go.Scatter(x=vte.index, y=vte.values, mode='lines+markers'))
fig.update_layout(title='Electric Vehicle Sales as a Percentage of Total Vehicle Sales',
                  xaxis_title='Date',
                  yaxis_title='Percentage of Total Vehicle Sales')
fig.show()

In [4]:
et = evc_df.groupby(['Fuel Type']).sum()
ete = et.loc['Electric'] / et.loc['All'] * 100

# overlay ete on fig plot
fig.add_trace(go.Scatter(x=ete.index, y=ete.values, mode='lines+markers'))
fig.update_layout(title='Electric Vehicle Sales as a Percentage of Total Vehicle Sales',
                  xaxis_title='Date',
                  yaxis_title='Percentage of Total Vehicle Sales')
fig.show()

In [5]:
pd.to_datetime(nte.index).year

NameError: name 'nte' is not defined

In [6]:
def convert_to_yearly_reporting(df):

SyntaxError: incomplete input (258844154.py, line 1)

In [16]:
pd.to_datetime(nte.index).dayofyear/365 + pd.to_datetime(nte.index).year - 2010

Index([ 12.58082191780818, 12.665753424657623, 12.747945205479482,
       12.832876712328698, 12.915068493150784,               13.0,
       13.084931506849216,  13.16164383561636, 13.246575342465803,
       13.328767123287662, 13.413698630136878, 13.495890410958964,
        13.58082191780818, 13.665753424657623, 13.747945205479482,
       13.832876712328698, 13.915068493150784],
      dtype='float64')

In [17]:
nte.values/100

array([0.00749819, 0.06325077, 0.08515552, 0.03870152, 0.0455442 ,
       0.04094977, 0.07686897, 0.09557466, 0.08125677, 0.0534304 ,
       0.08924138, 0.08538894, 0.08586731, 0.06482242, 0.09130209,
       0.05757566, 0.08744597])

In [7]:
nt = nsw_df.groupby(['Fuel Type']).sum().drop(columns=['State','Vehicle Type'])
nte = nt.loc['Electric']/nt.loc['All']*100

# overlay ete on fig plot
fig.add_trace(go.Scatter(x=nte.index, y=nte.values, mode='lines+markers'))
fig.update_layout(title='Electric Vehicle Sales as a Percentage of Total Vehicle Sales',
                  xaxis_title='Date',
                  yaxis_title='Percentage of Total Vehicle Sales')
fig.show()

display(nte)
# sum values of nte that are within the same year value of the datetime index
# display(nte.loc[nte.index.year==2020].sum())
# display(nte.loc[nte.index.year==2019].sum())
# display(nte.loc[nte.index.year==2018].sum())
[nte.loc[pd.to_datetime(nte.index).year==y].sum() for y in range(2018,2030)]

# smooth nte
# convert nte to a dataframe
nte_df = pd.DataFrame(nte)
# add a column for the year
nte_df['year'] = pd.to_datetime(nte_df.index).year
# groupby year and sum
nte_df = nte_df.groupby(['year']).sum()
# add a column for the year
nte_df['year'] = nte_df.index
# add a column for the smoothed values
nte_df['smoothed'] = nte_df.rolling(window=3).mean()
display(nte_df)


2022-07-31    0.749819
2022-08-31    6.325077
2022-09-30    8.515552
2022-10-31    3.870152
2022-11-30    4.554420
2022-12-31    4.094977
2023-01-31    7.686897
2023-02-28    9.557466
2023-03-31    8.125677
2023-04-30    5.343040
2023-05-31    8.924138
2023-06-30    8.538894
2023-07-31    8.586731
2023-08-31    6.482242
2023-09-30    9.130209
2023-10-31    5.757566
2023-11-30    8.744597
dtype: float64

ValueError: Cannot set a DataFrame with multiple columns to the single column smoothed